In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk.stem.porter as porter
from def_overlap_funcs import filter_words, similarity_score

In [2]:
stemmer = porter.PorterStemmer()

In [3]:
data = pd.read_csv('TLN-definitions-23.csv', sep=';')
data = data.applymap(lambda s: filter_words(set(word_tokenize(s))))
data = data.applymap(lambda tokens: {stemmer.stem(t) for t in tokens})

data

,door,ladybug,pain,blurriness
0,"{two, room, passag, use, construct, close, tem...","{insect, small, typic, spot, black, red, six, ...","{feel, mental, distress, physic}","{focu, sight}"
1,"{open, close}","{insect, cultur, mani, luck, wing, black, red,...","{hurt, physic, feel, bad, emot}","{absenc, shapeless, border, definit}"
2,"{door, object, let, hole, peopl, wall, two, en...","{insect, color, orang, black, red, dot, fli}","{fell, consciou, be, experi, suffer}","{ca, felt, border, clearli, sensat, someth, see}"
3,"{one, anoth, area, access, usabl}","{insect, small, red, back}","{live, describ, concept, suffer}","{sharp, lack}"
4,"{delimit, area, structur, access, allow}","{round, insect, fli, small}","{discomfort, feel, physic}","{clariti, characterist, precis, lack}"
5,"{object, use, block, passag, pass, move}","{insect, color, small, beatl, famili, tendenti...","{unpleas, sensat, burn}","{everyth, eye, sensat, blur, see}"
6,"{rotat, materi, push, made, wood, lock, handl,...","{insect, bring, small, luck, insectivor, pest,...","{felt, psycholog, sensat, creatur, natur, phys...","{inabl, whether, goal, conceptu, perceiv, phys..."
7,"{separ, object, wall, close, go, room, use, open}","{insect, cultur, bring, small, luck, black, re...","{caus, sad, injuri, pain, physic, feel, emot}","{vision, imag, eye, unfocus, memori, lack, bad}"
8,"{place, anoth, someth, open, access, order}","{insect, small, black, red, dot}","{sorrow, express, sentiment}","{clear, total, make, thant, see, bright, condit}"
9,"{access, room}","{insect, small, yellow, red, coat, dot}","{unpleas, sensat, result, exert, physic, emot}","{result, exert, physic, state, use, drug, conf..."


In [4]:
scores = []
words = list(data.columns.values)
for word in words:
    word_scores = [similarity_score(len(def1.intersection(def2)), len(def1), len(def2))
                   for def1 in data[word] for def2 in data[word] if def1 != def2]
    scores.append(word_scores)

In [5]:
[sum(word_scores) / len(word_scores) for word_scores in scores]

[0.23453733367526428,
 0.5967102020921473,
 0.2523792937586046,
 0.08098364598364605]